In [ ]:
# CÉLULA 1: INSTALAÇÃO DAS BIBLIOTERCAS
# -------------------------------------------------
# Bibliotecas atualizadas para usar um modelo gratuito do Google Generative AI (Gemini).
# Removemos bibliotecas específicas do Hugging Face que não serão mais usadas.
!pip install langchain langchain_experimental langchain-community pandas tabulate gdown langchain-google-genai google-ai-generativelanguage==0.6.15 -q

In [ ]:
# CÉLULA 2: CONFIGURAÇÃO DO TOKEN DE ACESSO DO GOOGLE GENERATIVE AI
# -------------------------------------------------
# Usaremos o Google Generative AI.
import os
# A biblioteca 'google.colab.userdata' é usada para gerenciar segredos de forma segura no Google Colab.
# É ALTAMENTE RECOMENDADO usar os "Secrets" do Colab para armazenar sua chave de API.
# 1. No painel esquerdo do Colab, clique no ícone de chave (Secrets).
# 2. Adicione um novo segredo com o nome 'GOOGLE_API_KEY' e o valor da sua chave de API.
# 3. Marque a caixa 'Notebook access' para este notebook.
# Depois, você pode carregar com:
from google.colab import userdata
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

# SE VOCÊ ESTÁ RECEBENDO "API KEY NOT VALID", A CHAVE ABAIXO ESTÁ INCORRETA.
# VÁ PARA https://aistudio.google.com/app/apikey PARA OBTER SUA CHAVE DE API VÁLIDA.
# SUBSTITUA EXATAMENTE "SUA_CHAVE_API_AQUI" PELA SUA CHAVE REAL.
#os.environ['GOOGLE_API_KEY'] = "GOOGLE_API_KEY" # <<<<<< SUBSTITUA PELA SUA CHAVE DE API AQUI!

In [ ]:
# CÉLULA 3: DOWNLOAD AUTOMÁTICO DO GOOGLE DRIVE E DESCOMPACTAÇÃO
# -------------------------------------------------
# Agora, vamos baixar o arquivo .zip diretamente de um link do Google Drive.
import gdown
import zipfile
import os
import re # Importando a biblioteca re para usar expressões regulares

# --- INSTRUÇÕES PARA O USUÁRIO ---
# 1. Faça o upload do arquivo '202401_NFs.zip' para o seu Google Drive.
# 2. Clique com o botão direito no arquivo > "Compartilhar" > "Compartilhar".
# 3. Em "Acesso geral", mude para "Qualquer pessoa com o link".
# 4. Clique em "Copiar link". O link será algo como:
#    https://drive.google.com/file/d/FILE_ID/view?usp=sharing
# 5. Copie o link completo e cole na variável abaixo. O código extrairá o ID.

# vvvv SUBSTITUA "SEU_LINK_COMPLETO_AQUI" PELO LINK COMPLETO DO SEU ARQUIVO vvvv
google_drive_link = "https://drive.google.com/file/d/1jaNl2kL-a32HO_jXH3g3J1mGHzHWm9YD/view?usp=sharing"
# ^^^^ SUBSTITUA "SEU_LINK_COMPLETO_AQUI" PELO LINK COMPLETO DO SEU ARQUIVO ^^^^

# Extrai o FILE_ID do link usando regex
match = re.search(r'/d/([a-zA-Z0-9_-]+)', google_drive_link)
google_drive_file_id = match.group(1) if match else None


zip_file_name = '202401_NFs.zip'
output_path = f'./{zip_file_name}'

# Constrói a URL de download direto e baixa o arquivo
if google_drive_file_id is None:
    print("⚠️ ATENÇÃO: Não foi possível extrair o ID do arquivo do Google Drive. Por favor, verifique se o link está correto.")
else:
    url = f'https://drive.google.com/uc?id={google_drive_file_id}'
    print(f"Baixando o arquivo de ID: {google_drive_file_id}...")
    gdown.download(url, output_path, quiet=False)

    # Verifica se o download foi bem-sucedido e descompacta
    if os.path.exists(zip_file_name):
        print(f"\nDownload concluído: {zip_file_name}")
        # Descompacta o arquivo
        with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
            zip_ref.extractall('dados_nfs') # Extrai para uma pasta chamada 'dados_nfs'
        print("\nArquivos descompactados com sucesso na pasta 'dados_nfs':")
        !ls dados_nfs
    else:
        print("\n❌ ERRO: O download do arquivo falhou. Verifique se o ID do arquivo está correto e se as permissões de compartilhamento estão como 'Qualquer pessoa com o link'.")

In [ ]:
# CÉLULA 4: CARREGAMENTO DOS DADOS COM PANDAS
# -------------------------------------------------
# Com os arquivos extraídos, vamos carregá-los em DataFrames do pandas.
# DataFrames são como planilhas inteligentes que podemos manipular com Python.
import pandas as pd

# Define os caminhos para os arquivos CSV
caminho_cabecalho = 'dados_nfs/202401_NFs_Cabecalho.csv'
caminho_itens = 'dados_nfs/202401_NFs_Itens.csv'

# Carrega os arquivos CSV
try:
    df_cabecalho = pd.read_csv(caminho_cabecalho, sep=',')
    df_itens = pd.read_csv(caminho_itens, sep=',')

    print("--- AMOSTRA DOS DADOS DO CABEÇALHO ---")
    print(df_cabecalho.head())
    print("\n--- AMOSTRA DOS DADOS DOS ITENS ---")
    print(df_itens.head())

except FileNotFoundError:
    print("\nERRO: Um ou ambos os arquivos CSV não foram encontrados. Verifique a extração.")


--- AMOSTRA DOS DADOS DO CABEÇALHO ---
                                CHAVE DE ACESSO  \
0  41240106267630001509550010035101291224888487   
1  50240129843878000170550010000025251000181553   
2  50240112977901000117550010000051831659469117   
3  35240161186490000157551000000170551435015613   
4  35240134028316923228550010003691801935917886   

                                              MODELO  SÉRIE   NÚMERO  \
0  55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...      1  3510129   
1  55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...      1     2525   
2  55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...      1     5183   
3  55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...    100    17055   
4  55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...      1   369180   

                              NATUREZA DA OPERAÇÃO         DATA EMISSÃO  \
0             Outras Entradas - Dev Remessa Escola  2024-01-18 07:10:39   
1               VENDA DE MERCADORIA FORA DO ESTADO  2024-01-26 11:24:42

In [ ]:
# CÉLULA 5: COMBINANDO OS DADOS
# -------------------------------------------------
# Para que o agente possa responder perguntas que cruzam informações
# (ex: "Qual o fornecedor do item mais caro?"), é melhor combinar
# os dois arquivos em um único DataFrame. Faremos isso usando a coluna
# 'CHAVE DE ACESSO' que existe em ambos.
df_completo = pd.merge(
    df_itens,
    df_cabecalho,
    on='CHAVE DE ACESSO', # Corrigido para usar a coluna comum
    how='left' # 'left' para manter todos os itens e adicionar as informações do cabeçalho
)

print("\n--- AMOSTRA DOS DADOS COMBINADOS ---")
# Mostra algumas colunas importantes do DataFrame combinado
print(df_completo[[
    'CHAVE DE ACESSO', # Corrigido para usar a coluna comum
    'RAZÃO SOCIAL EMITENTE_x', # Corrigido para usar o nome da coluna após o merge
    'DESCRIÇÃO DO PRODUTO/SERVIÇO', # Assumindo que 'DESCRIÇÃO DO PRODUTO/SERVIÇO' é a descrição do item
    'QUANTIDADE', # Assumindo que 'QUANTIDADE' é a quantidade do item
    'VALOR UNITÁRIO', # Assumindo que 'VALOR UNITÁRIO' é o valor unitário do item
    'VALOR NOTA FISCAL' # Assumindo que 'VALOR NOTA FISCAL' é o valor total da nota
]].head())

In [ ]:
# CÉLULA 6: CRIAÇÃO E CONFIGURAÇÃO DO AGENTE (COM GOOGLE GENERATIVE AI)
# -------------------------------------------------
# Esta é a parte principal! Vamos criar o agente usando um modelo gratuito do Google.
# Usaremos 'ChatGoogleGenerativeAI' para carregar um modelo de linguagem e
# o 'create_pandas_dataframe_agent' para conectá-lo aos nossos dados.
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_google_genai import ChatGoogleGenerativeAI
import os
import google.generativeai as genai # Importa a biblioteca do Google Generative AI

# Configura a chave de API diretamente com a biblioteca google.generativeai
genai.configure(api_key=os.environ.get('GOOGLE_API_KEY'))

# Inicializa o modelo de linguagem que o agente usará.
# ALTERADO para 'gemini-2.0-flash', um modelo mais recente e otimizado para velocidade/custo.
# O parâmetro 'temperature=0.1' torna as respostas mais diretas e factuais.
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.1)


# Cria o agente, passando o modelo (llm) e nosso DataFrame combinado (df_completo)
# 'verbose=True' faz com que o agente mostre o "raciocínio" dele ao responder.
agent = create_pandas_dataframe_agent(
    llm,
    df_completo,
    verbose=True,
    handle_parsing_errors=True, # Ajuda a lidar com possíveis erros de interpretação
    allow_dangerous_code=True # Permite que o agente execute código Python para analisar o DataFrame
)

print("\n✅ Agente criado com modelo do Google Generative AI e pronto para receber perguntas!")

In [ ]:

# CÉLULA 7: INTERFACE INTERATIVA PARA PERGUNTAS
# -------------------------------------------------
# Agora, vamos criar um loop para que você possa fazer perguntas
# ao agente continuamente. Para sair, digite 'sair'.

print("\n================================================")
print("🤖 ASSISTENTE DE ANÁLISE DE NOTAS FISCAIS 🤖")
print("================================================")
print("Faça perguntas sobre os dados das notas fiscais.")
print("Exemplos:")
print("  - Qual é o fornecedor que teve maior montante recebido no total?")
print("  - Qual item teve maior volume entregue em quantidade?")
print("  - Liste os 5 itens mais caros com base no valor unitário.")
print("  - Qual a data da nota fiscal mais recente?")
print("\nDigite 'sair' para encerrar.")
print("------------------------------------------------\n")

while True:
    pergunta_usuario = input("Sua pergunta: ")
    if pergunta_usuario.lower() == 'sair':
        print("Até logo!")
        break
    try:
        # Invoca o agente com a pergunta do usuário
        resposta = agent.invoke(pergunta_usuario)
        print("\nResposta do Agente:")
        # A resposta vem em um dicionário, o conteúdo está na chave 'output'
        print(resposta['output'])
        print("\n------------------------------------------------\n")
    except Exception as e:
        print(f"\nOcorreu um erro: {e}")
        print("Por favor, tente reformular sua pergunta.")
        print("\n------------------------------------------------\n")